In [4]:
import onnx
import os
import numpy as np
import onnxruntime as ort

In [8]:
import onnxruntime.backend as backend
import cv2

In [5]:
onnx_model = onnx.load("../models/detect/1/model.onnx")
onnx.checker.check_model(onnx_model)

In [9]:
def face_preparation(image_path, scales, resized_path):
    img = cv2.imread(image_path)
    print('Original shape:', img.shape)
    scales = scales

    im_shape = img.shape
    target_size = min(scales)
    max_size = max(scales)
    im_size_x = im_shape[1]
    im_size_y = im_shape[0]
    im_scale_x = float(scales[1]/im_size_x)
    im_scale_y = float(scales[0]/im_size_y)
    scales = [im_scale_x, im_scale_y]
    print('im_scale:',scales)

    resized_img = None
    if im_scale_x!=1.0 or im_scale_y!=1.0:
        resized_img = cv2.resize(img, None, None, fx=im_scale_x, fy=im_scale_y, interpolation=cv2.INTER_LINEAR)
        cv2.imwrite(resized_path, resized_img)
    else:
        resized_img = img.copy()

    print('Resized shape:', resized_img.shape)
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

    aligned = np.transpose(resized_img, (2,0,1)) #HWC->CHW

    return aligned, img, scales

In [40]:
def frame_preprocess(img, scales, resized_path):
    im_shape = img.shape
    target_size = scales[0]
    max_size = scales[1]
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])
    im_scale = float(target_size) / float(im_size_min)
    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    
    #print('im_scale:',scales)
    scales = [im_scale,im_scale]

    resized_img = None
    if im_scale!=1.0:
        resized_img = cv2.resize(img, None, None, fx=im_scale, fy=im_scale, interpolation=cv2.INTER_LINEAR)
        cv2.imwrite(resized_path, resized_img)
    else:
        resized_img = img.copy()

    print('Resized shape:', resized_img.shape)
    resized_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)

    dim_changed = np.transpose(resized_img, (2,0,1)) #HWC->CHW

    input_blob = np.expand_dims(dim_changed, axis=0).astype(np.float32)

    images_data = []
    # filenames = []
    # filename = unique_id
    images_data.append(input_blob)
    # filenames.append(filename)

    # return images_data, scales, im_shape
    return input_blob, scales, im_shape

In [33]:
scales = '900,900'
scales = scales.split(',')
scales = [int(scales[0]), int(scales[1])]
# im_size=[int(image_size.split(',')[0]), int(image_size.split(',')[1])]

In [34]:
scales

[900, 900]

In [35]:
image_path = './photos/altynbek.jpg'
img_name = './resized/' + image_path.split('/')[-1].split('.')[0] + '_resized.jpg'

In [36]:
img_name

'./resized/altynbek_resized.jpg'

In [22]:
align, result_img, im_scale = face_preparation(image_path, scales, img_name)

Original shape: (1280, 960, 3)
im_scale: [0.9375, 0.703125]
Resized shape: (900, 900, 3)


In [41]:
cv_img = cv2.imread(image_path)
blob_img, scale, im_shape = frame_preprocess(cv_img, scales, img_name)

Resized shape: (900, 675, 3)


In [42]:
def get_feature(onnx_file, input_blob, batching = False):
    # if not batching:
    #     # ONE INPUT
    #     input_blob = np.expand_dims(aligned, axis=0).astype(np.float32) #NCHW
    #     # ONE INPUT
    # else:
    #     # BATCHING
    #     input_blob = np.expand_dims(aligned, axis=0).astype(np.float32) #NCHW
    #     input_blob = np.squeeze(input_blob)
    #     # BATCHING
    onnx_model = onnx.load(onnx_file)
    ort_session = backend.prepare(onnx_model, 'GPU')
    outputs = ort_session.run(input_blob)

    return outputs

In [43]:
outputs = get_feature('../models/detect/1/model.onnx', blob_img)

EP Error /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1193 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_tensorrt.so with error: libnvinfer.so.8: cannot open shared object file: No such file or directory
 when using ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider']
Falling back to ['CUDAExecutionProvider', 'CPUExecutionProvider'] and retrying.


2023-12-17 01:56:44.572348042 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1,20,113,113} does not match actual shape of {1,20,113,85} for output face_rpn_landmark_pred_stride8
2023-12-17 01:56:44.659391685 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1,8,113,113} does not match actual shape of {1,8,113,85} for output face_rpn_bbox_pred_stride8
2023-12-17 01:56:45.011036123 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1,4,113,113} does not match actual shape of {1,4,113,85} for output face_rpn_cls_prob_reshape_stride8
2023-12-17 01:56:45.293606289 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1,20,57,57} does not match actual shape of {1,20,57,43} for output face_rpn_landmark_pred_stride16
2023-12-17 01:56:45.317452571 [W:onnxruntime:, execution_frame.cc:857 VerifyOutputSizes] Expected shape from model of {1,8,57,5

In [6]:
x, y = test_data[0][0], test_data[0][1]
ort_sess = ort.InferenceSession(onnx_model)
outputs = ort_sess.run(None, {'input': x.numpy()})

NameError: name 'test_data' is not defined